# [Minimum Obstacle Removal to Reach Corner](https://leetcode.com/problems/minimum-obstacle-removal-to-reach-corner/)

- **Given**: A grid `M`x`N`
    + 0: empty cell
    + 1: obstacle
- **Return**: the **minimum number of obstacles to remove** to walk from the upper left corner (0, 0) to the lower right corner (M-1, N-1)

#### Constraints

- $1 \leq M * N \leq 10^5$

#### Example 1

<img src="./assets/5.png" width="500"/>

```
Output: 2
Explanation: remove the obstacles at (0, 1) and (0, 2)
```

#### Example 2

<img src="./assets/6.png" width="400"/>

```
Output: 0
```

## Solution DP - $O(M^2*N^2)$ (TLE)

```Cpp
class Solution {
public:
    int minimumObstacles(vector<vector<int>> &A) {
        int N = A.size();
        int M = A[0].size();

        // Count max obstacles
        int max_obs = 0;
        for(int x=0; x<N; ++x) for(int y=0; y<M; ++y) if(A[x][y] == 1) max_obs += 1;

        // dp[x][y][k] = distance 
        vector<vector<vector<int>>> dp(N, vector<vector<int>>(M, vector<int>(max_obs + 1, INT_MAX)));
        dp[0][0][A[0][0]] = 0;

        queue<tuple<int,int,int>> Q;
        Q.push({0,0, A[0][0]});

        // BFS DP
        int ans = max_obs;
        while(!Q.empty()) {
            auto [x, y, k] = Q.front();
            Q.pop();

            if(x == N-1 && y == M-1) ans = min(ans, k);
            
            for(auto [x_, y_]: vector<pair<int,int>>{{0,1},{1,0},{0,-1},{-1,0}}) {
                int x_next = x + x_;
                int y_next = y + y_;

                if(0 <= x_next && x_next < N &&
                        0 <= y_next && y_next < M ) {
                    int k_next = A[x_next][y_next] == 1 ? k + 1 : k;

                    if(k_next <= max_obs && dp[x_next][y_next][k_next] > dp[x][y][k] + 1) {
                        dp[x_next][y_next][k_next] = dp[x][y][k] + 1;
                        Q.push( {x_next, y_next, k_next} );
                    }
                }
            }
        }
        return ans;
    }
};
```

## Solution PQ DP - $O(M*N*log(M*N))$

```Cpp
class Solution {
public:
    int minimumObstacles(vector<vector<int>> &A) {
        int N = A.size();
        int M = A[0].size();

        // dp[x][y] = optimal(k)
        vector<vector<int>> dp(N, vector<int>(M, INT_MAX));       
        dp[0][0] = A[0][0];

        // MinHeap[k] = {x, y}
        multimap<int, pair<int,int>> Q;
        Q.insert( {A[0][0], {0, 0}} );

        // PQ BFS
        while(Q.size() > 0) {
            auto [k, pos] = *Q.begin();
            auto [x, y] = pos;
            Q.erase(Q.begin());

            if(x == N-1 && y == M-1) return k;
            
            for(auto [x_, y_]: vector<pair<int,int>>{{0,1},{1,0},{0,-1},{-1,0}}) {
                int x_next = x + x_;
                int y_next = y + y_;

                if(0 <= x_next && x_next < N &&
                        0 <= y_next && y_next < M) {
                    int k_next = (A[x_next][y_next] == 1) ? k + 1 : k;

                    if(dp[x_next][y_next] > k_next) {
                        dp[x_next][y_next] = k_next;
                        Q.insert( {k_next, {x_next, y_next}} ); 
                    }
                }
            }
        }
        return -1;
    }
};
```

## Solution Deque - $O(M*N*(M+N))$
- **Idea**: Use a **deque** as PQ
    + high priority: push_front
    + low priority: push_back
    
```Cpp
class Solution {
public:
    int minimumObstacles(vector<vector<int>> &A) {
        int N = A.size();
        int M = A[0].size();

        // dp[x][y] = optimal(k)
        vector<vector<int>> dp(N, vector<int>(M, INT_MAX));       
        dp[0][0] = A[0][0];

        // deque = {k, x, y}
        deque<tuple<int,int,int>> D;
        D.push_front( {A[0][0], 0, 0} );

        // deque BFS
        while(D.size() > 0) {
            auto [k, x, y] = D.front();
            D.pop_front();

            if(x == N-1 && y == M-1) return k;
            
            for(auto [x_, y_]: vector<pair<int,int>>{{0,1},{1,0},{0,-1},{-1,0}}) {
                int x_next = x + x_;
                int y_next = y + y_;

                if(0 <= x_next && x_next < N &&
                        0 <= y_next && y_next < M &&
                        dp[x_next][y_next] == INT_MAX) {

                    // Encounter obs: less favor
                    if(A[x_next][y_next] == 1) {
                        dp[x_next][y_next] = dp[x][y] + 1;
                        D.push_back( {k+1, x_next, y_next} );
                    }

                    // Favor path with no obs
                    else {
                        dp[x_next][y_next] = dp[x][y];
                        D.push_front( {k, x_next, y_next} );
                    }
                }
            }
        }
        return -1;
    }
};
```